In [ ]:
from sklearn.neural_network import MLPClassifier
import pandas as pd

dataset_path = "../data_semeval/articles-ground-truth-validation-bypublisher-20181122.csv"
train_test_ratio = 0.75

data = pd.read_csv(dataset_path)
data["text"] = data["text"].apply(lambda x: x.replace("\n", " "))
first_test = round(train_test_ratio * len(data))

train_set = data.iloc[:first_test-1]
test_set = data.iloc[first_test:]

In [ ]:
import numpy as np
from gensim.sklearn_api import W2VTransformer

corpus = []
for article in data["text"]:
    corpus.append(article.split(" "))

def transform_article(article):
    wordvecs = model.transform(article.split(" "))
    return np.mean(wordvecs, axis=0)

model = W2VTransformer(size=150, min_count=1, seed=1)
model = model.fit(corpus)

train_x = np.array(list(train_set["text"].apply(transform_article)))
test_x = np.array(list(test_set["text"].apply(transform_article)))

train_y = list(train_set["hyperpartisan"].apply(lambda x: 1 if x else 0))
test_y = list(test_set["hyperpartisan"].apply(lambda x: 1 if x else 0))

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 3), random_state=1)
clf = clf.fit(train_x, train_y)

predicted_y = clf.predict(test_x)

In [ ]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tn = confusion_matrix(test_y, predicted_y, labels=None, sample_weight=None).ravel()
(tn, fp, fn, tn)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(test_y, predicted_y)